In [3]:
import re
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# The authentication
auth.authenticate_user()
from google.auth import default
credentials, _ = default()
docs_service = build('docs', 'v1', credentials=credentials)

# The input data
MEETING_NOTES = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

def parse_and_create_doc(title, md_input):
    """
    Parses markdown text and pushes formatted content to a new Google Doc.
    """
    try:
        # Create the new document
        doc_meta = {'title': title}
        new_doc = docs_service.documents().create(body=doc_meta).execute()
        document_id = new_doc.get('documentId')

        instruction_set = []
        lines = md_input.strip().split('\n')
        write_cursor = 1  # Google Docs index starts at 1

        for line in lines:
            if not line.strip() and not line.startswith('#'):
                continue

            # The default style and text transformations
            para_style = "NORMAL_TEXT"
            content = line + "\n"

            # The heading Level 1
            if line.startswith("# "):
                para_style = "HEADING_1"
                content = line[2:] + "\n"
            # The heading Level 2
            elif line.startswith("## "):
                para_style = "HEADING_2"
                content = line[3:] + "\n"
            # The heading Level 3
            elif line.startswith("### "):
                para_style = "HEADING_3"
                content = line[4:] + "\n"
            # The checkbox transformation
            elif "- [ ]" in line:
                content = line.replace("- [ ]", "☐") + "\n"
            # The list formatting (Bullet points)
            elif line.strip().startswith(("* ", "- ")):
                content = "• " + line.strip()[2:] + "\n"

            # 1. The insert Text Request
            instruction_set.append({
                'insertText': {
                    'location': {'index': write_cursor},
                    'text': content
                }
            })

            # 2. The paragraph Style Request
            instruction_set.append({
                'updateParagraphStyle': {
                    'range': {
                        'startIndex': write_cursor,
                        'endIndex': write_cursor + len(content)
                    },
                    'paragraphStyle': {'namedStyleType': para_style},
                    'fields': 'namedStyleType'
                }
            })

            # 3. The mention Styling (@name) - Dark Orchid Bold
            mentions = re.finditer(r'@\w+', content)
            for m in mentions:
                instruction_set.append({
                    'updateTextStyle': {
                        'range': {
                            'startIndex': write_cursor + m.start(),
                            'endIndex': write_cursor + m.end()
                        },
                        'textStyle': {
                            'bold': True,
                            'foregroundColor': {'color': {'rgbColor': {'red': 0.6, 'green': 0.2, 'blue': 0.6}}}
                        },
                        'fields': 'bold,foregroundColor'
                    }
                })

            # 4. The footer Styling
            if "Meeting recorded" in line or "Duration" in line:
                instruction_set.append({
                    'updateTextStyle': {
                        'range': {
                            'startIndex': write_cursor,
                            'endIndex': write_cursor + len(content)
                        },
                        'textStyle': {'italic': True, 'fontSize': {'magnitude': 9, 'unit': 'PT'}},
                        'fields': 'italic,fontSize'
                    }
                })

            write_cursor += len(content)

        # Batch execute all formatting instructions
        docs_service.documents().batchUpdate(
            documentId=document_id,
            body={'requests': instruction_set}
        ).execute()

        print(f"Document Generated Successfully!")
        print(f"URL: https://docs.google.com/document/d/{document_id}/edit")
        return document_id

    except HttpError as error:
        print(f"An API error occurred: {error}")
        return None

# Run the task
if __name__ == "__main__":
    parse_and_create_doc("Product Team Sync - May 15, 2023", MEETING_NOTES)

Document Generated Successfully!
URL: https://docs.google.com/document/d/1zwXMXdTLzKC9HefDICwfp6-tSPtCfhayVFL5wTjO-c0/edit
